In [3]:
import json
import pandas as pd 
import os 
from tqdm import tqdm
import re
from collections import defaultdict
from os import system

## generation the  send data

In [4]:
prompt_header = {
    'math': '你是一个数学老师，给定一道数学问题，你需要判断模型回答是否正确，分数在0到1之间。模型回答的最终结果一定要和标准答案中的最终结果一致。模型回答中的解题步骤可以和标准答案不同，但一定要是正确的。请按照"得分:"这样的形式输出分数。',
    'code': '你是一个计算机科学老师，给定一道编程问题，你需要判断模型回答能否解决该问题，分数在0到1之间。标准答案仅作为参考。模型回答中的代码步骤可以和标准答案不同，但一定要是正确的。请按照"得分:"这样的形式输出分数。',
    'rewrite': '你需要研究评价标准来对模型回答给出分数，满分为1分，最低分为0分。请按照"得分:"这样的形式输出分数 。评价标准要求模型回答能够满足问题中提出的指令。',  # no gold
    'classification': '你需要通过参考标准答案，来对模型回答给出分数，满分为1分，最低分为0分。请按照"得分:"这样的形式输出分数。评价标准要求模型回答和标准答案越接近越好。',
    'generation': '你需要研究评价标准来对模型回答给出分数，满分为1分，最低分为0分。请按照"得分:"这样的形式输出分数。评价标准要求模型回答语句通顺，符合问题要求，同时是真实且没有恶意的。',  # no gold
    'summarization': '你需要通过参考标准答案，来对模型回答给出分数，满分为1分，最低分为0分。请按照"得分:"这样的形式输出分数 。评价标准要求模型回答能包含输入文本信息的重点.',
    'extract': '你需要通过参考标准答案，来对模型回答给出分数，满分为1分，最低分为0分。请按照"得分:"这样的形式输出分数。评价标准要求模型抽取出来的结果来自问题中给出的上下文，并且很好的回答了该问题。',
    'open qa': '你需要通过参考标准答案，来对模型回答给出分数，满分为1分，最低分为0分。请按照"得分:"这样的形式输出分数。评价标准要求模型回答越接近标准答案分数越高。',
    'translation': '假设你是一个语言学家，你需要通过参考标准答案，来对模型回答给出分数，满分为1分，最低分为0分。请按照"得分:"这样的形式输出分数 。评价标准要求翻译过后的句子保持原有的意思，并且翻译过后的句子越通顺分数越高。',
    'brainstorming': '你需要研究评价标准来对模型回答给出分数，满分为1分，最低分为0分。请按照"得分:"这样的形式输出分数 。评价标准要求模型回答的内容对于问题有帮助，同时是真实且没有恶意的。',   # no gold
    'closed qa': '你需要通过参考标准答案，来对模型回答给出分数，满分为1分，最低分为0分。请按照"得分:"这样的形式输出分数。评价标准要求模型回答和标准答案越接近越好，且模型回答结果来自问题里面提供的信息。'   
}

In [5]:

inference_output_path = "data/send_test_bloom_7ep.jsonl"
test_path = 'eval_data/belle_test.test.json'
model_name = inference_output_path.split("/")[-1]
output_send_path= f"data/format_eval_{inference_output_path}"



In [6]:
'''Form test file send to chatgpt'''

# read test file
test_data = {}
with open(test_path, 'r') as f:
    for line in f:
        data = json.loads(line)
        test_data[data['id']] = data



unique_id = set()
with open(inference_output_path, 'r') as f:
    for line in f:
        data = json.loads(line)
        id= str(data['id'])
        if data['id'] not in unique_id:
                test_data[id]['model_response'] = data['response']
                unique_id.add(id)
assert len(test_data) == len(unique_id)

# combine for chatgpt evaluation 
send_data = []
for id in test_data:
    instruction = test_data[id]['instruction']
    gold_response = test_data[id]['gold_response']
    model_response = test_data[id]['model_response']
    cate = test_data[id]['type']
    if cate  in  ['rewrite','generation','brainstorming']:
        prompt = f'{prompt_header[cate]} 问题: {instruction} 模型回答: {model_response}' 
        # prompt = '{} 问题: {} 模型回答: {}'.format(prompt_header[cate], instruction, model_response)
    else:
        prompt = f'{prompt_header[cate]} 问题: {instruction} 标准答案: {gold_response} 模型回答: {model_response}'
        # prompt = '{} 问题: {} 标准答案: {} 模型回答: {}'.format(prompt_header[cate], instruction, gold_response, model_response)
    send_data.append({'id':id, 'prompt': prompt})

    
with open(f"data/send_data_{model_name}.txt",'w') as f:
    for data in send_data:
        f.write(json.dumps(data, ensure_ascii=False)+'\n')




## 解析分数

In [7]:
'''Read chatgpt returns and calculate score'''
def extract_score(text):
        pattern = r"\d+(?:\.\d+)?分"  # 匹配数字后面是“分”的字符串
        result = re.search(pattern, text)
        if result:
            return result.group().split("分")[0] # 返回提取结果
        else:
            pattern = r"得分:\d+(?:\.\d+)?"  # 匹配数字后面是“分”的字符串
            result = re.search(pattern, text)
            if result:
                return result.group().split("得分:")[1]  # 返回提取结果
            else:
                pattern = r"得分: \d+(?:\.\d+)?"  # 匹配数字后面是“分”的字符串
                result = re.search(pattern, text)
                if result:
                    return result.group().split("得分: ")[1]  # 返回提取结果
                else:
                    pattern = r"得分：\d+(?:\.\d+)?"  # 匹配数字后面是“分”的字符串
                    result = re.search(pattern, text)
                    if result:
                        return result.group().split("得分：")[1]  # 返回提取结果
                    else:
                    
                        pattern = r"得分为\d+(?:\.\d+)?"  # 匹配数字后面是“分”的字符串
                        result = re.search(pattern, text)
                        if result:
                            return result.group().split("得分为")[1]  # 返回提取结果                  

                    return None 

def gen_score_details_v2(input_path):
    lines = []
    cate2scores = defaultdict(list)
    allscorles = []
    parse_errors = []
    reflection = {'math': 'others', 'code': 'others', 'translation': 'rewrite'}
    with open(f'{input_path}', 'r') as f:
        for line in f:
            data = json.loads(line)
            id = data['data']['convId']
            # assert data['input']['convId'] == data['output']['data']['convId']
            prompt = data['data']['prompt']
            chatgpt_response = data['data']['responseMessage']
            score = extract_score(chatgpt_response)
            if score == None or (float(score) > 1 or float(score) < 0):
                # print(data)
                parse_errors.append(id)
                continue
            score = float(score)
            # previous info
            model_response = test_data[id]['model_response']
            instruction = test_data[id]['instruction']
            gold_response = test_data[id]['gold_response']
            cate = test_data[id]['type']
            cate = reflection.get(cate, cate)
            lines.append({'id':id, 'cate': cate, 'prompt': prompt, 'instruction': instruction, 'gold_response': gold_response, 'model_response': model_response, 'score': score, 'chatgpt_response': chatgpt_response})

            cate2scores[cate].append(score)
            allscorles.append(score)
    print (f'parse error: {len(parse_errors)}')
    cate2ave_score = {cate: sum(scores)/len(scores) for cate, scores in cate2scores.items()}
    overall_ave_score = sum(cate2ave_score.values())/len(cate2ave_score) # macro average
    overall_ave_score_wo_others = sum([cate2ave_score[cate] for cate in cate2ave_score if cate != 'others'])/(len(cate2ave_score)-1)  # macro average
    # overall_ave_score = sum(allscorles)/len(allscorles) # micro average
    cate2ave_score['macro_ave'] = overall_ave_score
    cate2ave_score['macro_ave_wo_others'] = overall_ave_score_wo_others
    cate2ave_score['model'] = model_name
    print (cate2ave_score)
    df = pd.DataFrame([cate2ave_score])
    column_order = ['model', 'others', 'rewrite', 'classification', 'generation', 'summarization', 'extract', 'open qa', 'brainstorming', 'closed qa', 'macro_ave', 'macro_ave_wo_others']
    df = df[column_order]
    # df.to_excel(f'{output_path}/score_{model_name}.xlsx', index=False)

    df_detail = pd.DataFrame.from_dict(lines, orient='columns')

    return df,df_detail




def gen_score_details_v1(input_path):
    lines = []
    cate2scores = defaultdict(list)
    allscorles = []
    parse_errors = []
    with open(f'{input_path}', 'r') as f:
        for line in f:
            data = json.loads(line)
            id = data['data']['convId']
            # assert data['input']['convId'] == data['output']['data']['convId']
            prompt = data['data']['prompt']
            chatgpt_response = data['data']['responseMessage']
            score = extract_score(chatgpt_response)
            if score == None or (float(score) > 1 or float(score) < 0):
                # print(data)
                parse_errors.append(id)
                continue
            score = float(score)
            # previous info
            model_response = test_data[id]['model_response']
            instruction = test_data[id]['instruction']
            gold_response = test_data[id]['gold_response']
            cate = test_data[id]['type']

            lines.append({'id':id, 'cate': cate, 'prompt': prompt, 'instruction': instruction, 'gold_response': gold_response, 'model_response': model_response, 'score': score, 'chatgpt_response': chatgpt_response})

            cate2scores[cate].append(score)
            allscorles.append(score)
    print (f'parse error: {len(parse_errors)}')
    cate2ave_score = {cate: sum(scores)/len(scores) for cate, scores in cate2scores.items()}
    overall_ave_score = sum(cate2ave_score.values())/len(cate2ave_score) # macro average
    overall_ave_score_wo_others = sum([cate2ave_score[cate] for cate in cate2ave_score if cate != 'code' and cate!='math'])/(len(cate2ave_score)-2)  # macro average
    # overall_ave_score = sum(allscorles)/len(allscorles) # micro average
    cate2ave_score['macro_ave'] = overall_ave_score
    cate2ave_score['macro_ave_no_code_math'] = overall_ave_score_wo_others
    cate2ave_score['model'] = model_name
    print (cate2ave_score)
    df = pd.DataFrame([cate2ave_score])
    column_order = ['model', 'math','code', 'rewrite', 'translation','classification', 'generation', 'summarization', 'extract', 'open qa', 'brainstorming', 'closed qa', 'macro_ave', 'macro_ave_no_code_math']
    df = df[column_order]
    # df.to_excel(f'{output_path}/score_{model_name}.xlsx', index=False)

    df_detail = pd.DataFrame.from_dict(lines, orient='columns')

    return df,df_detail



In [8]:
input_path="data/send_data_send_test_bloom_7ep.jsonl.txt.done"

score_df ,score_df_detailed=  gen_score_details_v2(input_path)

parse error: 2
{'open qa': 0.5038596491228068, 'brainstorming': 0.9332402234636875, 'others': 0.4672566371681416, 'summarization': 0.6512820512820515, 'classification': 0.7538461538461536, 'rewrite': 0.8061068702290073, 'generation': 0.944387755102041, 'closed qa': 0.49215686274509807, 'extract': 0.4756756756756757, 'macro_ave': 0.6697568754038515, 'macro_ave_wo_others': 0.6950694051833151, 'model': 'send_test_bloom_7ep.jsonl'}


In [9]:
score_df.head()

,model,others,rewrite,classification,generation,summarization,extract,open qa,brainstorming,closed qa,macro_ave,macro_ave_wo_others
0,send_test_bloom_7ep.jsonl,0.467257,0.806107,0.753846,0.944388,0.651282,0.475676,0.50386,0.93324,0.492157,0.669757,0.695069


In [10]:
score_df_detailed.head()

,id,cate,prompt,instruction,gold_response,model_response,score,chatgpt_response
0,3,open qa,"你需要通过参考标准答案，来对模型回答给出分数，满分为1分，最低分为0分。请按照""得分:""这样...",什么是机器学习？它有哪些应用场景？\n,机器学习是一种人工智能科学的分支，其目的是使计算机系统能够自动学习和改进，而不需要严格的编程...,机器学习是一种人工智能（AI）分支，旨在使计算机系统能够从数据中自动学习和改进。简而言之，它...,0.7,得分: 0.7
1,11,open qa,"你需要通过参考标准答案，来对模型回答给出分数，满分为1分，最低分为0分。请按照""得分:""这样...",解释什么是自然语言处理，列出至少两个自然语言处理应用场景。\n,自然语言处理（Natural Language Processing，NLP）是一种计算机科...,"自然语言处理（Natural Language Processing, NLP）是一种人工智...",0.8,得分: 0.8
2,6,brainstorming,"你需要研究评价标准来对模型回答给出分数，满分为1分，最低分为0分。请按照""得分:""这样的形式...",给出一份关于健康饮食的简短建议。\n,以下是一些关于健康饮食的简短建议：\n1. 多食用蔬菜和水果：蔬菜和水果富含维生素、矿物质和...,以下是一些简单而实用的健康饮食建议：\n1. 多吃蔬菜和水果：每天至少摄入5份新鲜蔬菜或水果...,1.0,得分: 1分。该回答清晰、简洁、准确，对问题给出了有效的健康饮食建议，没有恶意或误导性内容。
3,2,open qa,"你需要通过参考标准答案，来对模型回答给出分数，满分为1分，最低分为0分。请按照""得分:""这样...",回答以下问题的答案。\n中国有多少个省？,中国目前有34个省级行政区。具体分为23个省、5个自治区、4个直辖市和2个特别行政区。以下是...,截至2021年，中国的行政区划体系共有34个省级行政区。它们分别是：\n1. 北京市 - 是...,0.0,得分: 0分。模型回答虽然提到了中国有34个省级行政区，但并未给出完整的具体列表，也没有提到...
4,18,open qa,"你需要通过参考标准答案，来对模型回答给出分数，满分为1分，最低分为0分。请按照""得分:""这样...",回答以下问题：世界上最长的河流是什么？\n,世界上最长的河流是尼罗河（Nile River），它位于非洲大陆的东北部，涵盖了11个国家，...,世界上最长的河流是尼罗河。\n尼罗河位于非洲大陆，从布隆迪和卢旺达的尼亚萨湖流经肯尼亚、坦桑...,0.8,得分: 0.8
